# Level 2: Simple Agentic 

This tutorial is designed to give new users an overview of how to use llama-stack's builtin agent framework effectively. It demonstrates basic single-tool usage and provides guidance on switching between a local development environment and a remote kubernetes cluster.

## Overview

This tutorial walks you through how to build your own AI agent who can search the web following these steps:
1. Connecting to a llama-stack server (remote or local).
2. Configuring an agent for tool use.
3. Running the agent.


## Prerequisites

Before starting, ensure you have the following:
- Access to a remote cluster or a local Podman setup.
- API keys and user variables configured (e.g., `inference_model`, `TAVILY_SEARCH_API_KEY`, `LLAMA_STACK_ENDPOINT`).
- A Tavily API key is required. You can register for one at [https://tavily.com/](https://tavily.com/).


## Building a Web Search Agent
### 1. Connecting to a llama-stack server.
#### 1.1 Setting Up the Environment
- Import the necessary libraries.
- Defining user variables.
- Connection options
    - **Remote Server**: Set `remote=True` and provide the remote llama-stack endpoint.
    - **Local Server**: Set `remote=False` and provide the local llama-stack port (default: 8321).
    
    For detailed llama-stack server setup instructions, refer to:
    - [Remote Setup Guide](https://github.com/opendatahub-io/llama-stack-on-ocp/tree/main/kubernetes)
    - [Local Setup Guide](https://github.com/redhat-et/agent-frameworks/tree/main/prototype/frameworks/llamastack)

In [ ]:
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client import LlamaStackClient
from termcolor import cprint

inference_model = "ibm-granite/granite-3.2-8b-instruct"
TAVILY_SEARCH_API_KEY = "YOUR_TAVILY_API_KEY" # Replace with your tavily API key

remote = True # Use the `remote` variable to switching between a local development environment and a remote kubernetes cluster.
LLAMA_STACK_ENDPOINT = "your-llama-stack-endpoint" # Replace with your llama-stack endpoint if remote is set to True, or set it to default port 8321 if remote is set to False.

print(f"Model: {inference_model}")

Model: ibm-granite/granite-3.2-8b-instruct


#### 1.2 Client Initialization
- Initialize the `LlamaStackClient` with the appropriate base URL and provider data.  
- Connect the llama-stack client to the llama-stack server using the base URL.

In [2]:
base_url = LLAMA_STACK_ENDPOINT if remote else f"http://localhost:{LLAMA_STACK_ENDPOINT}"
client = LlamaStackClient(
        base_url=base_url,
        provider_data={"tavily_search_api_key": TAVILY_SEARCH_API_KEY}
)

### 2. Configuring an agent for tool use.
- **Agent Initialization**: 

- Create an `Agent` instance with the desired model, instructions, tools."

- **Instructions**: The `instructions` parameter, also referred to as the system prompt, specifies the agent's role and behavior. In this example, the agent is configured as a helpful web search assistant. It is instructed to use a tool whenever a web search is required and to respond in a friendly and helpful tone.

- **Tools**: The `tools` parameter defines the tools available to the agent. In this case, the `builtin::websearch` tool is used, which enables the agent to perform web searches. This tool is essential for retrieving up-to-date information from the web. Internally, it leverages Tavily Search to execute the search queries efficiently.

- **Tool Configuration**: The `tool_config` parameter allows for additional customization of the tools. For instance, the `"tool_choice": "required"` setting ensures that the agent must invoke the specified tool when performing a task. 

- **How It Works**: When a user query is provided, the agent processes the input and determines whether a tool is required to fulfill the request. If the query involves retrieving information from the web, the agent invokes the `builtin::websearch` tool. The tool interacts with Tavily Search to fetch real-time data, which is then processed and returned to the user in a friendly and helpful tone. This workflow ensures that the agent can handle a wide range of queries effectively.

For more details on the `builtin::websearch` tool and its capabilities, refer to the [Llama-stack tools documentation](https://llama-stack.readthedocs.io/en/latest/building_applications/tools.html). 

In [3]:
agent = Agent(
    client, 
    model=inference_model,
    instructions="""You are a helpful websearch assistant. When you are asked to search the web you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    tool_config={"tool_choice":"required"},
    sampling_params={"max_tokens":4096}
)

### 3. Running the agent.
- Define user prompts to interact with the agent.
- Use the agent to create a session and process user queries.
- Display the agent's responses for each query.

In [4]:
user_prompts = [
    "What’s latest in OpenShift?",
    "Search for recent news articles about advancements in quantum computing.",
    "Find coffee shops near Dublin city center that are open now, have Wi-Fi.",
]
for prompt in user_prompts:
    cprint(f"User> {prompt}", "blue")
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
    )
    for log in EventLogger().log(response):
        log.print()

User> What’s latest in OpenShift?
inference> <tool_call>
tool_execution> Tool:brave_search Args:{'query': 'latest updates on OpenShift'}
tool_execution> Tool:brave_search Response:{"query": "latest updates on OpenShift", "top_k": [{"title": "Red Hat unveils OpenShift 4.18: Enhanced security and virtualization ...", "url": "https://www.redhat.com/en/blog/what-you-need-to-know-red-hat-openshift-418", "content": "This article highlights the latest OpenShift 4.18 innovations and key enhancements. For a comprehensive list of updates and improvements, refer to the OpenShift 4.18 Release Notes. Enhance network flexibility with User Defined Networks and Border Gateway Protocol for pods and virtual machines", "score": 0.85567063, "raw_content": null}, {"title": "Experience the many enhancements in OpenShift 4.18", "url": "https://developers.redhat.com/blog/2025/02/25/whats-new-developers-red-hat-openshift-418", "content": "For an in-depth review of all of the latest features, including updates 

Here, we can observe that the `builtin::websearch` tool is used to perform a web search. The outputs are displayed in the notebook with color-coded text to help interpret the process:

- **Blue Text**: Represents the user's input or query.
- **Yellow Text**: Displays the LLM's inference response. 
- **Green Text**: Indicates the tool execution process, such as the tool being called and the query being sent to the web search API.

Great! We can see that the model returned some relevant and recent information about OpenShift. This was only possible due to its ability to call tools like the web search, demonstrating the agent's capacity to retrieve real-time data effectively.

## Key Takeaways
- This tutorial demonstrates how to set up and use a single tool with AI Agent.
- It highlights the flexibility of switching between remote and local environments.
- By following these steps, users can quickly get started with AI Agents and explore its capabilities.

For more advanced use cases, please check our other example jupyter notebooks.